# 1. Introduction

This notebook is for targeted variant analysis of KRAS in circulating tumor DNA (ctDNA) from a single patient (C_fw075). The raw data is from the NCBI BioProject PRJNA714799 related to the following publication:

O’Dea et al. Circulating tumor DNA sequencing in colorectal cancer patients treated with first-line chemotherapy with anti-EGFR. Scientific        Reports. 2021


You will find sections in the notebook for:

- processing raw paired-end ctDNA reads,
- align them to the human reference genome GRCh38
- perform variant calling with VarScan in the KRAS locus,
- annotate and interpret any the mutations in the KRAS region for clinically relevant alterations,
- briefly consider whether any new KRAS alterations emerge at follow-up.


# 2. Environment

All analyses were run on WSL2 UBUNTU on a personal (no organization affiliation) Windows 11 machine using a Conda environment named `ngs`. The environment contains common NGS tools:

`fastqc` – QC tool
`bowtie2` – aligner
`samtools` – BAM handling and mpileup generation  
`bcftools` – VCF utilities  
`varscan` – variant calling  
`snpEff` – variant annotation  

Python 3.11.14 (Jupyter in VSC) is used for:

light bookkeeping (paths, listing files),
parsing annotated VCFs and summarizing KRAS variants.
Text editing and summarizing.


In [1]:
#python imports

import pandas as pd
import os, glob
from pathlib import Path

In [30]:
# Define project root and all key subdirectories in one place

project_root = Path("/home/mgpersonal/ctdna_kras_project").resolve()

data_dir   = project_root / "data"
fastq_dir  = data_dir / "fastq"
ref_dir    = data_dir / "ref"
results_dir = project_root / "results"
bam_dir     = results_dir / "bam"
vcf_dir    = results_dir/ "vcf"
qc_dir  = results_dir/ "qc"
out_dir="${project_root}/results/summary"

# Existing Reference files (Moved a GRCh38 .fa and bowtie2 reference from my system to these directories)
ref_fasta  = ref_dir / "Homo_sapiens.GRCh38.dna.primary_assembly.fa"
ref_prefix  = ref_dir / "GRCh38_primary"  # Bowtie2 index prefix

# Create directory structure if it doesn't already exist
for d in [data_dir, fastq_dir, ref_dir, results_dir, bam_dir, vcf_dir]:
    d.mkdir(parents=True, exist_ok=True)

#Let's make sure everything is prese
print("project root:", project_root)
print("fastq_dir   :", fastq_dir)
print("ref_dir     :", ref_dir)
print("bam_dir     :", bam_dir)
print("vcf_dir     :", vcf_dir)
print("Reference fasta:", ref_fasta)
print("Bowtie2 prefix:", ref_prefix)


project root: /home/mgpersonal/ctdna_kras_project
fastq_dir   : /home/mgpersonal/ctdna_kras_project/data/fastq
ref_dir     : /home/mgpersonal/ctdna_kras_project/data/ref
bam_dir     : /home/mgpersonal/ctdna_kras_project/results/bam
vcf_dir     : /home/mgpersonal/ctdna_kras_project/results/vcf
Reference fasta: /home/mgpersonal/ctdna_kras_project/data/ref/Homo_sapiens.GRCh38.dna.primary_assembly.fa
Bowtie2 prefix: /home/mgpersonal/ctdna_kras_project/data/ref/GRCh38_primary


# 3. Data Acquisition

I selected C_fw075 after reviewing the runinfo.csv in the repository because it had 1 baseline and 1 follow-up cell free dna sample.

Our pre-treatment sample is `SRR13973824` → sample label `C_fw075_baseline`
Our post or follow-up treatment sample `SRR13973825` → sample label `C_fw075_followup`

The raw data were obtained from the NCBI SRA project associated with the paper.  
In practice, I pulled the data using `prefetch` and `fasterq-dump` (only run once on my machine). An example of the commands is:



In [ ]:
###First we need to inspect runinfo on NCBI repository to select a subject C_fw pre and post sample(follow-up)
#I just downloaded this file when I was browsing and moved the .csv into the sra directory.

runinfo_path = "/home/mgpersonal/ctdna_kras_project/data/sra/PRJNA714799_runinfo.csv"
runinfo = pd.read_csv(runinfo_path)


runinfo.head()

###Had to manually inspect/confirm the SampleName column has the information we want.


,Run,ReleaseDate,LoadDate,spots,bases,spots_with_mates,avgLength,size_MB,AssemblyName,download_path,...,Affection_Status,Analyte_Type,Histological_Type,Body_Site,CenterName,Submission,dbgap_study_accession,Consent,RunHash,ReadHash
0,SRR13974039,2022-04-18 00:00:24,2021-03-16 08:45:24,2132268,615326977,2053223,288,350,GCA_000001405.13,https://sra-downloadb.be-md.ncbi.nlm.nih.gov/s...,...,NaN,NaN,NaN,NaN,SEOUL NATIONAL UNIVERSITY,SRA1206760,NaN,public,25A5EE99E44E4FBC0D6B95BE1221A156,5A979126CE22F541C18C1B3DB7394495
1,SRR13974041,2022-04-18 00:00:24,2021-03-16 08:45:29,2704436,789609381,2632994,291,420,GCA_000001405.13,https://sra-downloadb.be-md.ncbi.nlm.nih.gov/s...,...,NaN,NaN,NaN,NaN,SEOUL NATIONAL UNIVERSITY,SRA1206760,NaN,public,099ADF641FDB3EC474D711584108DBB1,86C1109112E35E3F10C96AFAB8216EC2
2,SRR13974040,2022-04-18 00:00:24,2021-03-16 08:45:31,2263227,659576113,2218251,291,354,GCA_000001405.13,https://sra-downloadb.be-md.ncbi.nlm.nih.gov/s...,...,NaN,NaN,NaN,NaN,SEOUL NATIONAL UNIVERSITY,SRA1206760,NaN,public,C85AA55C37FB24B12E19C3B70DC7EC67,CBF81247DE6651D5F9633ACB7E61A8A4
3,SRR13974043,2022-04-18 00:00:24,2021-03-16 08:47:07,944806,265589338,900493,281,141,GCA_000001405.13,https://sra-downloadb.be-md.ncbi.nlm.nih.gov/s...,...,NaN,NaN,NaN,NaN,SEOUL NATIONAL UNIVERSITY,SRA1206760,NaN,public,B07E7B03747F8B6788FECE06D2052AB7,01C130C3118CF67695B55B2D77782834
4,SRR13974042,2022-04-18 00:00:24,2021-03-16 08:46:13,1374004,389796958,1360349,283,182,GCA_000001405.13,https://sra-downloadb.be-md.ncbi.nlm.nih.gov/s...,...,NaN,NaN,NaN,NaN,SEOUL NATIONAL UNIVERSITY,SRA1206760,NaN,public,8BE2BA608B12D72AEFFB1939015AFFF7,23C18760023C61F4C910D0CDA575CE91


In [4]:
###Identify the samples and the Run ID SRR
samples = runinfo["SampleName"].isin(["C_fw075", "C_fw075-2"])
ctdna = runinfo.loc[samples, ["Run", "SampleName"]]
ctdna


,Run,SampleName
208,SRR13973824,C_fw075
209,SRR13973825,C_fw075-2


In [37]:
%%bash
set -euo pipefail

project_root="/home/mgpersonal/ctdna_kras_project"
fastq_dir="${project_root}/data/fastq"
tmp_dir="${project_root}/data/sra_tmp"

mkdir -p "${fastq_dir}" "${tmp_dir}"

for srr in SRR13973824 SRR13973825; do
  fasterq-dump \
    --split-3 \
    --temp "${tmp_dir}" \
    -O "${fastq_dir}" \
    "${srr}"
done


spots read      : 4,998,783
reads read      : 9,997,566
reads written   : 9,905,463
spots read      : 5,508,730
reads read      : 11,017,460
reads written   : 10,946,198


In [ ]:

%%bash
set -euo pipefail

#need for shell
project_root="/home/mgpersonal/ctdna_kras_project"
fastq_dir="${project_root}/data/fastq"

for srr in SRR13973824 SRR13973825; do
  wc -l "${fastq_dir}/${srr}_1.fastq" "${fastq_dir}/${srr}_2.fastq"
done

##I was having an issue with different reads coming out of a 2 split so I made it a 3 split in the previous cell and now R1 and R2 work for pre and follow-up sample
#this just verifies R1 and R2 are the same for bowtie2.

  19626720 /home/mgpersonal/ctdna_kras_project/data/fastq/SRR13973824_1.fastq
  19626720 /home/mgpersonal/ctdna_kras_project/data/fastq/SRR13973824_2.fastq
  39253440 total
  21749872 /home/mgpersonal/ctdna_kras_project/data/fastq/SRR13973825_1.fastq
  21749872 /home/mgpersonal/ctdna_kras_project/data/fastq/SRR13973825_2.fastq
  43499744 total


# 4. QC - FASTQC assessment 

Sequenced quality was assessed for the C_fw075 subject's pre and follow-up forward and reverse fastq files. Read quality, GC content, and adapter content all appeared good. It didn't look like there was any striking adapter or issues with falloff at the tails.

I decided not to perform any trimming because of the decent quality (> 30 fred). I will certainly reassess if I see anything concerning downstream. It looks like the authors of the paper do have a "filtered" file in their methods, but I couldn't find the exact method and I did not want to get too fixed on this for the puprose and scope of the assignment. 

In [39]:
###Check fastqs (not included) to make sure we have all pairs.
##We can just keep the singletons in the .fastq

fastqs = sorted(glob.glob(os.path.join(fastq_dir, "*.fastq*")))
fastqs


['/home/mgpersonal/ctdna_kras_project/data/fastq/SRR13973824.fastq',
 '/home/mgpersonal/ctdna_kras_project/data/fastq/SRR13973824_1.fastq',
 '/home/mgpersonal/ctdna_kras_project/data/fastq/SRR13973824_2.fastq',
 '/home/mgpersonal/ctdna_kras_project/data/fastq/SRR13973825.fastq',
 '/home/mgpersonal/ctdna_kras_project/data/fastq/SRR13973825_1.fastq',
 '/home/mgpersonal/ctdna_kras_project/data/fastq/SRR13973825_2.fastq']

In [20]:
###Running fASTQC for Read Quality Assessment
#Asess raw read quality for the baseline and follow-up ctDNA fastq files

# Find all FASTQ files in fastq_dir
fastqs = sorted(glob.glob(os.path.join(fastq_dir, "*.fastq")))

for f in fastqs:
    print("  -", os.path.basename(f))

# Need to add a space for FASTQC command to work
fastqs_str = " ".join(fastqs)

# Run FastQC
!fastqc -o {qc_dir} {fastqs_str}




  - SRR13973824.fastq
  - SRR13973824_1.fastq
  - SRR13973824_2.fastq
  - SRR13973825.fastq
  - SRR13973825_1.fastq
  - SRR13973825_2.fastq
null
null
Started analysis of SRR13973824.fastq
null
null
null
null
Approx 5% complete for SRR13973824.fastq
Approx 10% complete for SRR13973824.fastq
Approx 15% complete for SRR13973824.fastq
Approx 20% complete for SRR13973824.fastq
Approx 25% complete for SRR13973824.fastq
Approx 30% complete for SRR13973824.fastq
Approx 35% complete for SRR13973824.fastq
Approx 40% complete for SRR13973824.fastq
Approx 45% complete for SRR13973824.fastq
Approx 50% complete for SRR13973824.fastq
Approx 55% complete for SRR13973824.fastq
Approx 60% complete for SRR13973824.fastq
Approx 65% complete for SRR13973824.fastq
Approx 70% complete for SRR13973824.fastq
Approx 75% complete for SRR13973824.fastq
Approx 80% complete for SRR13973824.fastq
Approx 85% complete for SRR13973824.fastq
Approx 90% complete for SRR13973824.fastq
Approx 95% complete for SRR13973824.f

5. Analysis

Paired-reads of the baseline and follow-up samples are aligned to the GRCh38 reference genome using bowtie2. The sorted.bams are indexed and we perform variant calling with VarScan mpileup2cn restricted to the KRAS region chr12:25245300–25245400. The VCF parameters were very flexible to try to detect any small fraction mutation that might appear, or even a de-novo alteration. The VCF files for baseline and follow-up samples were annotated with SnipEff (VEP and ClinVar would be nice, but they were not working well in my environment. COSMIC is unfortanately not available for personal non-affiliated use) There was evidence of the clinically meaningful mutation KRAS G12D present in the baseline sample but not the follow-up sample suggesting the metastatic CRC is responding to cetuximab treatment.

Below are the key finding and commentary:

1. Baseline Sample: Clear Detection of Oncogenic KRAS G12D

In the baseline sample (C_fw075_baseline), we observe a well-supported KRAS G12D mutation (cDNA change 35G>A; Protein change Gly12Asp):

Position: chr12:25245350
Reference / Alt: C>T (GRCh38)
Depth: ~1,300× total coverage
ALT allele count: ~110 reads
VAF: ~8.3%
Annotation: missense_variant, impact: MODERATE

KRAS G12D is a well-known hotspot mutation and a common driver of several cancers including CRC. In the baseline sample it is present at  about 8% VAF. Since this is from circulating DNA it likely would have a smaller VAF. Importantly, the mutation is supported by high depth and can be considered a robust call.

2. Follow-up Sample: Absence of KRAS G12D and No High-Confidence Driver Mutations

In the follow-up sample (C_fw075_followup), no G12D mutation is detected, even at low allele frequencies. Instead, the only KRAS-related variant observed is:

Position: chr12:25250747
Change: G>A
Effect: splice_region_variant & intron_variant
Depth: 46×
ALT allele count: 2 
VAF: 4%
Annotation impact: LOW

Even though this variant has an LOW annotation impact from SnpEFF, there is not too much known about it. It is also at a very low depth and VAF. For these 2 reasons it can be considered a weak call, if not background. The absence of the KRAS G12D alteration is likely due to cetuximab treatment response. It can be a bit reassuring to the patient that the levels of that oncogenic driver are not detectable post treatment.

It is also good to see no new robust KRAS alteration appear like G12C, A, R, etc. even though this is not the usual resistance mechanism to cetuximab. 

In [40]:
###Review the FASTQC .hmtl files
#It looks like the sequences are pretty good quality (>30) in all the files. Sure the adapters and tails fall off a bit, but since I am less familiar with this exact targetted sequencing, I am going to avoid trimming unless something looks concerning down the line.
qc_dir = "/home/mgpersonal/ctdna_kras_project/results/qc"
sorted(glob.glob(os.path.join(qc_dir, "*fastqc.html")))

['/home/mgpersonal/ctdna_kras_project/results/qc/SRR13973824_1_fastqc.html',
 '/home/mgpersonal/ctdna_kras_project/results/qc/SRR13973824_2_fastqc.html',
 '/home/mgpersonal/ctdna_kras_project/results/qc/SRR13973824_fastqc.html',
 '/home/mgpersonal/ctdna_kras_project/results/qc/SRR13973825_1_fastqc.html',
 '/home/mgpersonal/ctdna_kras_project/results/qc/SRR13973825_2_fastqc.html',
 '/home/mgpersonal/ctdna_kras_project/results/qc/SRR13973825_fastqc.html']

In [ ]:
%%bash
set -euo pipefail

#need to def these again for this bash shell
project_root="/home/mgpersonal/ctdna_kras_project"
fastq_dir="${project_root}/data/fastq"
ref_dir="${project_root}/data/ref"
bam_dir="${project_root}/results/bam"

ref_prefix="${ref_dir}/GRCh38_primary"


sra_id="SRR13973824"
label="C_fw075_baseline"

R1="${fastq_dir}/${sra_id}_1.fastq"
R2="${fastq_dir}/${sra_id}_2.fastq"

echo "Aligning ${sra_id} (${label})..."
bowtie2 -x "${ref_prefix}" \
  -1 "${R1}" -2 "${R2}" \
  -p 8 \
  2> "${bam_dir}/${label}.bowtie2.log" \
| samtools sort -o "${bam_dir}/${label}.sorted.bam"

#.sorted.bam created for pre sample

Aligning SRR13973824 (C_fw075_baseline)...
ERROR! Session/line number was not unique in database. History logging moved to new session 17


[bam_sort_core] merging from 4 files and 1 in-memory blocks...


In [ ]:
%%bash
set -euo pipefail

#need to def these again for this bash shell
project_root="/home/mgpersonal/ctdna_kras_project"
fastq_dir="${project_root}/data/fastq"
ref_dir="${project_root}/data/ref"
bam_dir="${project_root}/results/bam"

ref_prefix="${ref_dir}/GRCh38_primary"

#change these for follow-up
sra_id="SRR13973825"
label="C_fw075_followup"

R1="${fastq_dir}/${sra_id}_1.fastq"
R2="${fastq_dir}/${sra_id}_2.fastq"

echo "Aligning ${sra_id} (${label})..."
bowtie2 -x "${ref_prefix}" \
  -1 "${R1}" -2 "${R2}" \
  -p 8 \
  2> "${bam_dir}/${label}.bowtie2.log" \
| samtools sort -o "${bam_dir}/${label}.sorted.bam"

#.sorted.bam created for follow-up sample

Aligning SRR13973825 (C_fw075_followup)...


[bam_sort_core] merging from 5 files and 1 in-memory blocks...


In [ ]:
%%bash
set -euo pipefail

bam_dir="/home/mgpersonal/ctdna_kras_project/results/bam"
cd "${bam_dir}"

for bam in *.sorted.bam; do
    echo "Indexing $bam"
    samtools index "$bam"
done

echo "Done."


#we are just indexing our .sorted.bams here

Indexing C_fw075_baseline.sorted.bam


Indexing C_fw075_followup.sorted.bam
Done.


In [ ]:
%%bash
set -euo pipefail

project_root="/home/mgpersonal/ctdna_kras_project"
ref_dir="${project_root}/data/ref"
bam_dir="${project_root}/results/bam"
vcf_dir="${project_root}/results/vcf"

ref_fasta="${ref_dir}/Homo_sapiens.GRCh38.dna.primary_assembly.fa"
# looked up GRCh38 KRAS coordinates on Ensembl 12: 25,205,246-25,250,936 reverse strand. Padded the region slightly.
region="12:25204000-25253000"

# Let's confirm our sorted sorted.bams
echo "BAMs:"
ls -lh "${bam_dir}"/*.sorted.bam || true

# loop over pre and follow-up sample creating mpileup straight to VCF file of KRAS region
for label in C_fw075_baseline C_fw075_followup; do
    bam="${bam_dir}/${label}.sorted.bam"
    mpileup="${vcf_dir}/${label}.kras.mpileup"
    vcf="${vcf_dir}/${label}.kras.varscan.vcf"

    echo
    echo "mpileup + VarScan for ${label}"
    echo "  BAM: ${bam}"
    echo "  Region: ${region}"

    # mpileup restricted to KRAS region
    samtools mpileup -f "${ref_fasta}" -r "${region}" "${bam}" > "${mpileup}"

    # VarScan consensus 
    #
    varscan mpileup2cns "${mpileup}" \
        --min-coverage 10 \
        --min-avg-qual 25 \
        --min-var-freq 0.01 \
        --p-value 0.99 \
        --output-vcf 1 > "${vcf}"

    echo "  vcf created: ${vcf}"
done

echo
echo "Done."


BAMs:
-rw-r--r-- 1 mgpersonal mgpersonal 846M Nov 15 14:38 /home/mgpersonal/ctdna_kras_project/results/bam/C_fw075_baseline.sorted.bam
-rw-r--r-- 1 mgpersonal mgpersonal 900M Nov 15 14:51 /home/mgpersonal/ctdna_kras_project/results/bam/C_fw075_followup.sorted.bam

mpileup + VarScan for C_fw075_baseline
  BAM: /home/mgpersonal/ctdna_kras_project/results/bam/C_fw075_baseline.sorted.bam
  Region: 12:25204000-25253000


[mpileup] 1 samples in 1 input files
/home/mgpersonal/miniconda3/envs/ngs/bin/varscan: line 6: warning: setlocale: LC_ALL: cannot change locale (en_US.UTF-8): No such file or directory
Min coverage:	10
Min reads2:	2
Min var freq:	0.01
Min avg qual:	25
P-value thresh:	0.99
Reading input from /home/mgpersonal/ctdna_kras_project/results/vcf/C_fw075_baseline.kras.mpileup
9037 bases in pileup file
28 variant positions (21 SNP, 7 indel)
5 were failed by the strand-filter
28 variant positions reported (21 SNP, 7 indel)


  vcf created: /home/mgpersonal/ctdna_kras_project/results/vcf/C_fw075_baseline.kras.varscan.vcf

mpileup + VarScan for C_fw075_followup
  BAM: /home/mgpersonal/ctdna_kras_project/results/bam/C_fw075_followup.sorted.bam
  Region: 12:25204000-25253000


[mpileup] 1 samples in 1 input files
/home/mgpersonal/miniconda3/envs/ngs/bin/varscan: line 6: warning: setlocale: LC_ALL: cannot change locale (en_US.UTF-8): No such file or directory
Min coverage:	10
Min reads2:	2
Min var freq:	0.01
Min avg qual:	25
P-value thresh:	0.99
Reading input from /home/mgpersonal/ctdna_kras_project/results/vcf/C_fw075_followup.kras.mpileup
8233 bases in pileup file
20 variant positions (13 SNP, 7 indel)
2 were failed by the strand-filter
20 variant positions reported (13 SNP, 7 indel)


  vcf created: /home/mgpersonal/ctdna_kras_project/results/vcf/C_fw075_followup.kras.varscan.vcf

Done.


In [26]:
%%bash
set -euo pipefail
cd ~/ctdna_kras_project/results/vcf

# baseline
snpEff -Xmx4g -v GRCh38.99 \
  C_fw075_baseline.kras.varscan.vcf \
  > C_fw075_baseline.kras.ann.vcf

# follow-up
snpEff -Xmx4g -v GRCh38.99 \
  C_fw075_followup.kras.varscan.vcf \
  > C_fw075_followup.kras.ann.vcf



00:00:00 SnpEff version SnpEff 5.3a (build 2025-09-02 10:24), by Pablo Cingolani
00:00:00 Command: 'ann'
00:00:00 Reading configuration file 'snpEff.config'. Genome: 'GRCh38.99'
00:00:00 Looking for config file: '/home/mgpersonal/ctdna_kras_project/results/vcf/snpEff.config'
00:00:00 Reading config file: /home/mgpersonal/miniconda3/envs/ngs/share/snpeff-5.3.0a-1/snpEff.config
00:00:00 done
00:00:00 Reading database for genome version 'GRCh38.99' from file '/home/mgpersonal/miniconda3/envs/ngs/share/snpeff-5.3.0a-1/./data/GRCh38.99/snpEffectPredictor.bin' (this might take a while)
00:00:13 done
00:00:13 Loading Motifs and PWMs
00:00:13 Building interval forest
00:00:17 done.
00:00:17 Genome stats :
#-----------------------------------------------
# Genome name                : 'Homo_sapiens'
# Genome version             : 'GRCh38.99'
# Genome ID                  : 'GRCh38.99[0]'
# Has protein coding info    : true
# Has Tr. Support Level info : true
# Genes                      : 60676


In [27]:

%%bash
set -euo pipefail
cd ~/ctdna_kras_project/results/vcf
pwd
ls -lh


/home/mgpersonal/ctdna_kras_project/results/vcf
total 192M
-rw-r--r-- 1 mgpersonal mgpersonal 1.2M Nov 15 15:01 C_fw075_baseline.kras.ann.vcf
-rw-r--r-- 1 mgpersonal mgpersonal    0 Nov 14 21:42 C_fw075_baseline.kras.clinvar.tsv
-rw-r--r-- 1 mgpersonal mgpersonal 5.7M Nov 15 15:00 C_fw075_baseline.kras.mpileup
-rw-r--r-- 1 mgpersonal mgpersonal 460K Nov 14 22:29 C_fw075_baseline.kras.summary.tsv
-rw-r--r-- 1 mgpersonal mgpersonal 1.2M Nov 15 15:00 C_fw075_baseline.kras.varscan.vcf
-rw-r--r-- 1 mgpersonal mgpersonal 1.2M Nov 15 15:01 C_fw075_followup.kras.ann.vcf
-rw-r--r-- 1 mgpersonal mgpersonal 7.2M Nov 15 15:00 C_fw075_followup.kras.mpileup
-rw-r--r-- 1 mgpersonal mgpersonal 420K Nov 14 22:29 C_fw075_followup.kras.summary.tsv
-rw-r--r-- 1 mgpersonal mgpersonal 1.2M Nov 15 15:00 C_fw075_followup.kras.varscan.vcf
-rw-r--r-- 1 mgpersonal mgpersonal 173M Nov  9 14:34 clinvar.vcf.gz
-rw-r--r-- 1 mgpersonal mgpersonal 592K Nov  9 14:34 clinvar.vcf.gz.tbi
-rw-r--r-- 1 mgpersonal mgpersonal

In [ ]:
##Make parsed summary tables from the annotated .vcfs

baseline_vcf = os.path.join(vcf_dir, "C_fw075_baseline.kras.ann.vcf")
followup_vcf  = os.path.join(vcf_dir, "C_fw075_followup.kras.ann.vcf")


# Parse the SnpEff-annotated VCF and keep only KRAS variants that are missense/nonsense/splice-related and have a non-empty IMPACT field.
def extract_kras_coding(vcf_path):
 
    rows = []
    with open(vcf_path) as f:
        for line in f:
            if line.startswith("#"):
                continue

            parts = line.rstrip("\n").split("\t")
            if len(parts) < 10:
                continue

            chrom, pos, vid, ref, alt, qual, flt, info, fmt, sample = parts

            # Find ANN=... in INFO
            ann = None
            for kv in info.split(";"):
                if kv.startswith("ANN="):
                    ann = kv.split("=", 1)[1]
                    break
            if ann is None:
                continue

            # Loop over each ANN entry. We just want effect, impact, gene, cdna, and protein to represent our findins.
            for entry in ann.split(","):
                fields = entry.split("|")
                if len(fields) < 11:
                    continue

                effect  = fields[1]   
                impact  = fields[2]   
                gene    = fields[3]   
                cdna    = fields[9]   
                protein = fields[10]  

                # Only KRAS
                if gene != "KRAS":
                    continue

                # Require impact annotation
                if impact in ("", "."):
                    continue

                # Only missense / nonsense / splice-related
                effect_lower = effect.lower()
                if not (
                    "missense" in effect_lower
                    or "nonsense" in effect_lower
                    or "splice"   in effect_lower
                ):
                    continue

                # Parse FORMAT from VarScan info
                fmt_keys = fmt.split(":")
                fmt_vals = sample.split(":")
                fmt_dict = dict(zip(fmt_keys, fmt_vals))

                rd = fmt_dict.get("RD")
                ad = fmt_dict.get("AD")
                dp = fmt_dict.get("DP")
                freq = fmt_dict.get("FREQ")

                try:
                    rd_int = int(rd)
                except:
                    rd_int = np.nan

                try:
                    ad_int = int(ad)
                except:
                    ad_int = np.nan

                try:
                    dp_int = int(dp)
                except:
                    dp_int = rd_int + ad_int if not np.isnan(rd_int) else np.nan

                vaf = np.nan
                if freq and freq.endswith("%"):
                    try:
                        vaf = float(freq.rstrip("%"))
                    except:
                        vaf = np.nan

                rows.append({
                    "CHROM": chrom,
                    "POS": int(pos),
                    "REF": ref,
                    "ALT": alt,
                    "REF_COUNT": rd_int,
                    "ALT_COUNT": ad_int,
                    "DEPTH": dp_int,
                    "VAF_percent": vaf,
                    "cDNA_change": cdna,
                    "protein_change": protein,
                    "effect": effect,
                    "impact": impact,
                })

    df = pd.DataFrame(rows)

    # drop transcripts for summary
    if not df.empty:
        df = (
            df.drop_duplicates(
                subset=["CHROM", "POS", "REF", "ALT", "protein_change", "effect", "impact"]
            )
            .reset_index(drop=True)
        )

    return df

# Use the simpler extractor
baseline_kras = extract_kras_coding(baseline_vcf)
followup_kras = extract_kras_coding(followup_vcf)

print("Baseline KRAS coding/splice variants with impact:")
display(baseline_kras)

print("\nFollow-up KRAS coding/splice variants with impact:")
display(followup_kras)


Baseline KRAS coding/splice variants with impact:


,CHROM,POS,REF,ALT,REF_COUNT,ALT_COUNT,DEPTH,VAF_percent,cDNA_change,protein_change,effect,impact
0,12,25245350,C,T,1201,110,1315,8.37,c.35G>A,p.Gly12Asp,missense_variant,MODERATE



Follow-up KRAS coding/splice variants with impact:


,CHROM,POS,REF,ALT,REF_COUNT,ALT_COUNT,DEPTH,VAF_percent,cDNA_change,protein_change,effect,impact
0,12,25250747,G,A,44,2,46,4.35,c.-12+4C>T,,splice_region_variant&intron_variant,LOW


6. Ideas/Vision Board for visualizations for marketing or presentations

With a variant calling analysis a lot of times its hard to avoid a polished summary table that shows relevant information like the Basline/follow-up tables above which ended our analysis. It is likely important for shareholders within the organization to see information like the depth and VAF, since these scientist understand all this information. For marketing to a physician or medical group it may be enough to just show the VAF. That being said, there are several figures that may communicate the table information well or support the table.

1) In my analysis I only had a single post treatment or follow-up sample, but some patients had up to 4 that I saw. A nice longitudinal plot with the x axis the dates of sample procurement and y axis being the VAF may look nice. It could communicate response of the metastatic CRC to the cetuximab over those sample time points, or if it pops up again, perhaps resistance. Either way, showing an important hotspot mutation like KRAS G12D pre or post cetuximab would likely lead to getting the patient on a type 1 KRAS inhibitor. 

2) Imaging the KRAS alterations with IGV could be meaningful to internal shareholders and scientists since they understand more of the techanical and conceptual aspect of the NGS. I did not image this bam with IGV but from the depth of the KRAS G12D snp would would look striking. 

3) It may be nice for internal shareholders and marketing to show a nice EGFR and downstream MAPK pathway proteins. If we need to speak specifically about KRAS we could show how normal KRAS signals and how altered KRAS signals. It certainly would be nice to see where all the panel proteins are positioned in the signaling. 

